In [9]:
from pyspark.sql import SparkSession
from pyspark.streaming import context

In [2]:
spark = SparkSession.builder.appName("stream").getOrCreate()

24/04/24 18:24:03 WARN Utils: Your hostname, alber-victus resolves to a loopback address: 127.0.1.1; using 10.251.208.55 instead (on interface wlp4s0)
24/04/24 18:24:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/24 18:24:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
lines = spark.readStream.format('socket').option('host','localhost').option("port",8888).load()

24/04/24 18:25:45 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
